In [20]:
import yfinance as yf
import pandas as pd
from currency_converter import CurrencyConverter

In [21]:
tickers = ["NOK", "REMEDY.HE", "KEMPOWR.HE", "LMT", "RHM.DE", "SAAB-B.ST", "GOOG", "WITH.HE"] #Add the tickers of your portfolio
fileName = tickers
c = CurrencyConverter()
ROUND_TO_BILLIONS = 0.000001

In [22]:
# Create an empty DataFrame to store the information for all tickers
df = pd.DataFrame()


In [23]:
# Loop through each ticker and retrieve the information
for symbol in tickers:
    stock = yf.Ticker(ticker=symbol)
    stock_info = stock.get_info()
    
    # Filter out unwanted values from the dictionary and append to the DataFrame
    wanted_values = ['website', 'longName', 'industry', 'sector', 'fullTimeEmployees', 'auditRisk', 'boardRisk', 'previousClose', 'open', 'dayLow', 'dayHigh', 'dividendRate', 'dividendYield', 'payoutRatio', 'trailingPE', 'forwardPE', 'marketCap', 'priceToSalesTrailing12Months', 'enterpriseValue', 'profitMargins', 'priceToBook', 'trailingEps', 'freeCashflow']
    #Array for rounded values
    rounded_values = ['marketCap', 'enterpriseValue', 'freeCashflow']
    #converted values
    convert_to_euros = ['previousClose', 'marketCap', 'open', 'dayLow', 'dayHigh', 'dividendRate', 'freeCashflow'] 
    
    d = {}
    for k, v in stock_info.items():
        if k in wanted_values:
            if k in rounded_values:
                v *= ROUND_TO_BILLIONS
            elif k in convert_to_euros and stock_info.get('currency') == 'SEK':
                v = round(c.convert(v, 'SEK', 'EUR'),1) * ROUND_TO_BILLIONS
            elif k == 'enterpriseValue':
                v*= ROUND_TO_BILLIONS
            d[k] = [v]    
    df_temp = pd.DataFrame.from_dict(data=d, orient='columns')
    df_temp['symbol'] = symbol  # Add a column for the ticker symbol
    df = pd.concat([df, df_temp], ignore_index=True) 

In [43]:
# Colorize the columns
styles = [
    {'selector': 'th', 'props': [('background-color', 'red'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center')]},
    {'selector': 'tr:nth-child(odd)', 'props': [('background-color', 'pink'), ('text-align', 'center'), ('font-weight', 'bold')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', 'white'), ('color', 'black'), ('text-align', 'center'), ('font-weight', 'bold')]},
]
df_styled = df.style.set_table_styles(styles)

# Display the styled DataFrame
df_styled

,website,industry,sector,fullTimeEmployees,auditRisk,boardRisk,previousClose,open,dayLow,dayHigh,dividendRate,dividendYield,payoutRatio,trailingPE,forwardPE,marketCap,priceToSalesTrailing12Months,enterpriseValue,profitMargins,priceToBook,trailingEps,longName,freeCashflow,symbol
0,https://www.nokia.com,Communication Equipment,Technology,86896,1.000000,3.000000,4.010000,4.050000,4.050000,4.100000,0.120000,0.028800,0.104100,4.897591,8.295918,22813.999104,0.897412,20055.113728,0.169810,1.063858,0.830000,Nokia Oyj,206.000000,NOK
1,https://www.remedygames.com,Electronic Gaming & Multimedia,Communication Services,334,nan,nan,24.950000,25.000000,24.700000,25.100000,0.100000,0.003900,nan,nan,-833.333400,336.752512,7.725808,285.682560,-0.039600,3.805175,-0.130000,Remedy Entertainment Oyj,0.510125,REMEDY.HE
2,https://kempower.com,Electrical Equipment & Parts,Industrials,375,nan,nan,34.000000,34.320000,33.400000,35.360000,nan,nan,0.000000,558.333400,67.000000,1856.513024,17.912455,1798.145792,0.034280,18.457300,0.060000,Kempower Oyj,-14.113637,KEMPOWR.HE
3,https://www.lockheedmartin.com,Aerospace & Defense,Industrials,116000,4.000000,6.000000,455.750000,457.030000,452.860000,458.270000,12.000000,0.026600,0.531400,21.054323,17.576418,115332.677632,1.743608,126548.762624,0.085990,11.991916,21.630000,Lockheed Martin Corporation,4343.625216,LMT
4,https://www.rheinmetall.com/en/rheinmetall_ag/home.php,Aerospace & Defense,Industrials,22227,2.000000,2.000000,263.200000,263.500000,261.100000,264.100000,4.300000,0.016300,0.374400,29.659866,14.034335,11348.653056,1.744069,12000.616448,0.073000,3.823499,8.820000,Rheinmetall AG,104.125000,RHM.DE
5,https://www.saabgroup.com,Aerospace & Defense,Industrials,19651,3.000000,8.000000,0.000055,0.000055,0.000053,0.000055,0.000000,0.008800,0.276000,29.979963,23.717798,79387.934720,1.793146,76304.007168,0.058120,2.652800,19.960000,Saab AB (publ),4922.625024,SAAB-B.ST
6,https://www.abc.xyz,Internet Content & Information,Communication Services,190711,6.000000,2.000000,123.520000,124.200000,123.029900,126.479000,nan,nan,0.000000,26.519398,19.719553,1557948.399616,5.473938,1456822.550528,0.205850,6.000390,4.640000,Alphabet Inc.,55862.624256,GOOG
7,https://www.withsecure.com,Software—Infrastructure,Technology,1245,2.000000,5.000000,1.513000,1.514000,1.496000,1.527000,nan,nan,0.000000,nan,-74.800000,263.106000,1.912511,220.214192,3.073930,2.038147,-0.210000,WithSecure Oyj,-6.796250,WITH.HE


In [44]:
# Save the DataFrame to a CSV file with a filename based on the tickers
csv_file = '_'.join(tickers)
df.to_csv(f'{tickers}.csv', index=False)